# Historical weather status for Saudi Arabia cities
_Auther: Esraa Madi_


-------

Planning a vacation, trip, party or other events? Then you need to know how the weather affects your event.

The weather status is important as well for road safety, it affects directly the number of road accidents. The emergency department in every hospital takes in their consideration the weather status as well, they have a peak in their patients number in the seasonal time.

So knowing the weather status is important for several aspects of our life. Checking the weather forecast is easy if try it for next Tuesday, but what if you are planning a big outdoor wedding for spring 2022 or your class reunion for next winter? 

Here we are trying to collect historical weather patterns for Saudi Arabia to be able to use it later in forecasting weather conditions that you can expect for any location and at any date, even one far into the future.

-----------

For the weather status, we collect following info:
- Date
- Time
- Temperature
- Weather description ( clear - sunny - .... )
- Wind speed
- Humidity
- Barometer (atmospheric pressure)
- Visibility (how much be able to see or be seen)

-----

We use this [weather website](https://www.timeanddate.com/weather/) to collect hourly weather data from **2017** to **2019** for all kingdom main cities:
- Qassim
- Hail
- Madina
- EP
- Riyadh
- Mecca
- Tabuk
- Assir
- Northern boarder
- Jazan
- Najran
- Baha
- Jawf

----------

In [1]:
# import libraries
import requests
import pandas as pd
from time import sleep

In [2]:
# import web scraping libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

In [3]:
# set the main URL for scaping
root_url = 'https://www.timeanddate.com/weather/{}/historic?month={}&year={}'

----

**To be able to request a city weather data, we need to add 3 parameters on the root URL:**
- City name or code: on our weather website, there is a name or code for each city. you should use it  during the search on the website to get city weather data (I have collected them manually from the site)
- Month: it would be (1 to 12)
- Year: it would be (2017 to 2019)

----

In [5]:
# dictionary for all cities name with corresponding code or name on the requested URL
cities_url = {'Qassim': '@108932' ,
              'Hail': '@106281',
              'Madina': 'saudi-arabia/medina',
              'EP': 'saudi-arabia/dammam',
              'Riyadh': 'saudi-arabia/riyadh' ,
              'Mecca': 'saudi-arabia/makkah',
              'Tabuk': 'saudi-arabia/tabuk',
              'Assir':  'saudi-arabia/khamis-mushait',
              'Northern boarder': 'saudi-arabia/rafha',
              'Jazan': '@10972356',
              'Najran': '@103631',
              'Baha': '@109953',
              'Jawf': '@109468'}

In [4]:
# create a webdriver object to start crawling on the weather website
# we used here chrome driver 
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

In [6]:
# create empty dataframe to store collected weather data
df = pd.DataFrame()

In [8]:
## loop over all cities
for city, url in cities_url.items(): 
    
    # loop over all years
    for year in [2017, 2018, 2019]:
        
        # loop over all months
        for month in range(1,13):

            # set up the city URL with 3 required parameters
            driver.get(root_url.format(url, month, year))
            
            # retrieve drop-down list for available days
            select_fr = Select(driver.find_element_by_id("wt-his-select"))

            # loop over all days
            for ind, day in enumerate(select_fr.options):

                select_fr.select_by_index(ind)
                #sleep(0.5)
                
                # retrieve html page 
                HTML = driver.page_source
                
                # create beautiful soup object to start parsing html tags and get required info
                soup = BeautifulSoup(HTML, 'html.parser')
                
                # retrieve main table of weather status
                table = soup.find_all(name='tbody')[1].find_all(name='tr')
                
                # loop over hours
                for hour in table:
                    
                    # create an empty list to be a row in the dataframe
                    row = []
                    
                    # save city, year, month, day in our row list
                    row.extend([city, year, month, day.text])
                    
                    # save other weather information (temperature, weather desc, wind, humidity, barometer, visibility)
                    row.append(hour.find(name='th').text[:5])
                    for info in hour.find_all(name='td'):
                        row.append(info.text)
                        
                    # add collected data for this hour to our dataframe
                    df = df.append([row])


In [30]:
# assign columns name
df.columns= ['city', 'year', 'month', 'date', 'time', 'temp', 'weather', 'wind', 'humidity', 'barometer', 'visibility']

In [31]:
# check first 5 rows in our dataframe
df.head()

,city,year,month,date,time,temp,weather,wind,humidity,barometer,visibility
0,Qassim,2017.0,1.0,1 January 2017,00:00,17 °C,Clear.,11 km/h,64%,1018 mbar,16 km
1,Qassim,2017.0,1.0,1 January 2017,01:00,17 °C,Clear.,6 km/h,64%,1018 mbar,16 km
2,Qassim,2017.0,1.0,1 January 2017,03:00,15 °C,Clear.,11 km/h,72%,1019 mbar,16 km
3,Qassim,2017.0,1.0,1 January 2017,04:00,15 °C,Clear.,11 km/h,72%,1019 mbar,16 km
4,Qassim,2017.0,1.0,1 January 2017,05:00,15 °C,Clear.,9 km/h,72%,1019 mbar,16 km


In [10]:
# check dataframe size
df.shape

(249023, 12)

In [32]:
# check columns types
df.dtypes

city           object
year          float64
month         float64
date           object
time           object
temp           object
weather        object
wind           object
humidity       object
barometer      object
visibility     object
dtype: object

In [12]:
# save the dataframe as csv file 
df.to_csv('weather-sa-2017-2019.csv')

> After finishing the scraping part, we have 249023 recorded hourly weather status. As we can see above our collected data needs some cleaning such as removing special characters and change columns type and so on.

In [29]:
# re-load the data from the file 
df = pd.read_csv('weather-sa-2017-2019.csv', index_col=0)

In [33]:
# change year, month columns type to integer
df['year'] = df['year'].astype('int')
df['month'] = df['month'].astype('int')

In [34]:
# check dataframe after fixing year, month types
df.head(3)

,city,year,month,date,time,temp,weather,wind,humidity,barometer,visibility
0,Qassim,2017,1,1 January 2017,00:00,17 °C,Clear.,11 km/h,64%,1018 mbar,16 km
1,Qassim,2017,1,1 January 2017,01:00,17 °C,Clear.,6 km/h,64%,1018 mbar,16 km
2,Qassim,2017,1,1 January 2017,03:00,15 °C,Clear.,11 km/h,72%,1019 mbar,16 km


In [35]:
# create day column
df['day'] = df['date'].map(lambda x: int(x.split(' ')[0]))

In [37]:
# create hour column 
df['hour'] = df['time'].map(lambda x: int(x.split(':')[0]))
df['hour'] = df['hour'].map(lambda x: 24 if x == 0 else x)

In [39]:
# create minutes column
df['minute'] = df['time'].map(lambda x: int(x.split(':')[1]))

In [40]:
# remove '°C' from temp column and fix its type
df.temp = df.temp.map(lambda x: x.replace('\xa0',' '))
df.temp = df.temp.map(lambda x: int(x.split(' ')[0]))

In [41]:
# remove '.' from weather column
df.weather = df.weather.map(lambda x: x.replace('.',' '))

In [42]:
# check dataframe after fixing previous columns
df.head(3)

,city,year,month,date,time,temp,weather,wind,humidity,barometer,visibility,day,hour,minute
0,Qassim,2017,1,1 January 2017,00:00,17,Clear,11 km/h,64%,1018 mbar,16 km,1,24,0
1,Qassim,2017,1,1 January 2017,01:00,17,Clear,6 km/h,64%,1018 mbar,16 km,1,1,0
2,Qassim,2017,1,1 January 2017,03:00,15,Clear,11 km/h,72%,1019 mbar,16 km,1,3,0


In [43]:
# check missing values on wind column
df.wind.isnull().sum()

109

In [44]:
# fill missing values with '-1 km/h'
df.wind.fillna('-1 km/h', inplace=True)

In [46]:
# replace 'No wind' with '0 km/h' to be able to change column type to integer
df.wind = df.wind.map(lambda x: str(x).replace('No wind','0 km/h'))
df.wind = df.wind.map(lambda x: int(str(x).split(' ')[0]))

In [47]:
# check missing values on humidity column
df.humidity.isnull().sum()

17

In [ ]:
# fill missing values with '-1%'
df.humidity.fillna('-1%', inplace=True)

In [ ]:
# remove '%' from humidity column
df.humidity = df.humidity.map(lambda x: int(str(x).replace('%','')))

In [48]:
# remove 'mbar' from barometer column
df.barometer = df.barometer.map(lambda x: str(x).split(' ')[0])

In [49]:
# check missing values on visibility column
df.visibility.isnull().sum()

49644

In [50]:
# fill missing values with '-1 km'
df.visibility.fillna('-1 km', inplace=True)

In [51]:
# remove 'km' from visibility column
df.visibility = df.visibility.map(lambda x: int(str(x).split('km')[0]))

In [52]:
# check dataframe after fixing previous columns
df.head(3)

,city,year,month,date,time,temp,weather,wind,humidity,barometer,visibility,day,hour,minute
0,Qassim,2017,1,1 January 2017,00:00,17,Clear,11,64%,1018,16,1,24,0
1,Qassim,2017,1,1 January 2017,01:00,17,Clear,6,64%,1018,16,1,1,0
2,Qassim,2017,1,1 January 2017,03:00,15,Clear,11,72%,1019,16,1,3,0


In [53]:
# change the order of columns
df = df[['city', 'date', 'time', 'year', 'month', 'day', 'hour', 'minute', 'weather', 'temp', 'wind', 'humidity', 'barometer', 'visibility']]

In [54]:
# check dataframe after fixing previous columns
df.head(3)

,city,date,time,year,month,day,hour,minute,weather,temp,wind,humidity,barometer,visibility
0,Qassim,1 January 2017,00:00,2017,1,1,24,0,Clear,17,11,64%,1018,16
1,Qassim,1 January 2017,01:00,2017,1,1,1,0,Clear,17,6,64%,1018,16
2,Qassim,1 January 2017,03:00,2017,1,1,3,0,Clear,15,11,72%,1019,16


In [55]:
# save dataframe after cleaning
df.to_csv('weather-sa-2017-2019-clean.csv')

----------

**You can find the clean data on [kaggle](https://www.kaggle.com/esraamadi/saudi-arabia-weather-history)**